In [8]:
library(dplyr)   
library(caret)
library(tidyverse)
library(recipes)
library(pROC)
library(caTools)
library(Metrics)
library(rsq)

options(repr.plot.width = 20, repr.plot.height =10, repr.plot.res = 600)


load(file = "C:/Users/abonini/Desktop/Analysis/Sources/LIVERTRANSPLANT/Modello/df_num_finale.Rdata")

df_num <- df_num_finale

# Feature selection

In [9]:
library(leaps)

Best_Subset <-
    regsubsets(degenzaPostTraOSP~.,
               data =df_num,
               nbest = 1,      # 1 best model for each number of predictors
               nvmax = NULL,    # NULL for no limit on number of variables
               force.in = NULL, force.out = NULL,
               method = "exhaustive")
summary_best_subset <- summary(Best_Subset)

which.max(summary_best_subset$adjr2)

[1] 15

In [10]:
tmp <- summary_best_subset$which[18,]

tmp

(Intercept)                      age            degenzaPreTra 
                    TRUE                    FALSE                     TRUE 
           ratioPaO2FiO2         platCountTranspl                 albumina 
                    TRUE                     TRUE                     TRUE 
         surgeryDuration IntraoperativTransfected             plasmaFresco 
                    TRUE                    FALSE                     TRUE 
             plasmaPools           doseFibrinogen         CRIOprecipitated 
                    TRUE                     TRUE                     TRUE 
        CristalloidiComp           Albumina50Comp   noradrenalineEndSurger 
                    TRUE                     TRUE                     TRUE 
        intermedLactates             coldIschemia            organDonorAge 
                    TRUE                    FALSE                     TRUE 
        organDonorHeight organDonorDrawalSodiemia        organDonorGammaGT 
                    TRUE                     TRUE                     TRUE 
              realMELDNa 
                    TRUE

In [11]:
df_num_select <- subset(df_num, select= c(degenzaPreTra,ratioPaO2FiO2,platCountTranspl,albumina,surgeryDuration,plasmaFresco,plasmaPools,doseFibrinogen,CRIOprecipitated,CristalloidiComp,Albumina50Comp,noradrenalineEndSurger,intermedLactates,organDonorAge,organDonorHeight,organDonorDrawalSodiemia,organDonorGammaGT,realMELDNa,degenzaPostTraOSP))

In [12]:
dim(df_num_select)

[1] 1396   19

# Data partition

In [13]:
#make this example reproducible
set.seed(1)

#use 70% of dataset as training set and 30% as test set
sample <- sample.split(df_num$degenzaPostTraOSP, SplitRatio = 0.7)
train  <- subset(df_num, sample == TRUE)
test   <- subset(df_num, sample == FALSE)


In [14]:
#make this example reproducible
set.seed(1)

#use 70% of dataset as training set and 30% as test set
sample_select <- sample.split(df_num_select$degenzaPostTraOSP, SplitRatio = 0.7)
train_select  <- subset(df_num_select, sample_select == TRUE)
test_select   <- subset(df_num_select, sample_select == FALSE)


# Model

In [15]:
ctrl <- trainControl(method = "repeatedcv", 
                 number = 10, 
                 savePredictions = "final",verboseIter=TRUE, search= "random") 

#summaryFunction = multiClassSummary (to keep all metrics)

In [16]:
lm <- train(degenzaPostTraOSP ~ ., data = train,
               method = "lm",
               trControl = ctrl)

+ Fold01.Rep1: intercept=TRUE 
- Fold01.Rep1: intercept=TRUE 
+ Fold02.Rep1: intercept=TRUE 
- Fold02.Rep1: intercept=TRUE 
+ Fold03.Rep1: intercept=TRUE 
- Fold03.Rep1: intercept=TRUE 
+ Fold04.Rep1: intercept=TRUE 
- Fold04.Rep1: intercept=TRUE 
+ Fold05.Rep1: intercept=TRUE 
- Fold05.Rep1: intercept=TRUE 
+ Fold06.Rep1: intercept=TRUE 
- Fold06.Rep1: intercept=TRUE 
+ Fold07.Rep1: intercept=TRUE 
- Fold07.Rep1: intercept=TRUE 
+ Fold08.Rep1: intercept=TRUE 
- Fold08.Rep1: intercept=TRUE 
+ Fold09.Rep1: intercept=TRUE 
- Fold09.Rep1: intercept=TRUE 
+ Fold10.Rep1: intercept=TRUE 
- Fold10.Rep1: intercept=TRUE 
Aggregating results
Fitting final model on full training set


In [17]:
lm_select <- train(degenzaPostTraOSP ~ ., data = train_select,
               method = "lm",
               trControl = ctrl)

+ Fold01.Rep1: intercept=TRUE 
- Fold01.Rep1: intercept=TRUE 
+ Fold02.Rep1: intercept=TRUE 
- Fold02.Rep1: intercept=TRUE 
+ Fold03.Rep1: intercept=TRUE 
- Fold03.Rep1: intercept=TRUE 
+ Fold04.Rep1: intercept=TRUE 
- Fold04.Rep1: intercept=TRUE 
+ Fold05.Rep1: intercept=TRUE 
- Fold05.Rep1: intercept=TRUE 
+ Fold06.Rep1: intercept=TRUE 
- Fold06.Rep1: intercept=TRUE 
+ Fold07.Rep1: intercept=TRUE 
- Fold07.Rep1: intercept=TRUE 
+ Fold08.Rep1: intercept=TRUE 
- Fold08.Rep1: intercept=TRUE 
+ Fold09.Rep1: intercept=TRUE 
- Fold09.Rep1: intercept=TRUE 
+ Fold10.Rep1: intercept=TRUE 
- Fold10.Rep1: intercept=TRUE 
Aggregating results
Fitting final model on full training set


In [18]:
lm

Linear Regression 

979 samples
 21 predictor

No pre-processing
Resampling: Cross-Validated (10 fold, repeated 1 times) 
Summary of sample sizes: 881, 881, 882, 880, 881, 881, ... 
Resampling results:

  RMSE       Rsquared   MAE      
  0.6293783  0.2040544  0.4662144

Tuning parameter 'intercept' was held constant at a value of TRUE

In [19]:
lm_select

Linear Regression 

979 samples
 18 predictor

No pre-processing
Resampling: Cross-Validated (10 fold, repeated 1 times) 
Summary of sample sizes: 881, 880, 882, 883, 880, 881, ... 
Resampling results:

  RMSE       Rsquared   MAE      
  0.6281836  0.2010027  0.4643716

Tuning parameter 'intercept' was held constant at a value of TRUE

In [20]:
# collect resamples
results <- resamples(list(lm=lm,  lm_select=lm_select))
# summarize the distributions
summary(results)
# boxplots of results
bwplot(results)
# dot plots of results
dotplot(results)


Call:
summary.resamples(object = results)

Models: lm, lm_select 
Number of resamples: 10 

MAE 
              Min.   1st Qu.    Median      Mean   3rd Qu.      Max. NA's
lm        0.405684 0.4458561 0.4713261 0.4662144 0.4915278 0.5133325    0
lm_select 0.387230 0.4432469 0.4683170 0.4643716 0.4899039 0.5227195    0

RMSE 
               Min.   1st Qu.    Median      Mean   3rd Qu.      Max. NA's
lm        0.5246697 0.5799191 0.6159030 0.6293783 0.6960167 0.7250824    0
lm_select 0.5331011 0.5869737 0.6388036 0.6281836 0.6755429 0.7077910    0

Rsquared 
                Min.   1st Qu.    Median      Mean   3rd Qu.      Max. NA's
lm        0.07992698 0.1571374 0.2046806 0.2040544 0.2588337 0.2939171    0
lm_select 0.05645533 0.1329535 0.1963352 0.2010027 0.2567859 0.3683004    0


In [21]:
vimp <- varImp(lm_select)
vimp
plot(vimp, cex= 1.5,  xlab = list( cex = 1.4), ylab = list( cex = 1.4), scales = list(x = list(cex=1.1),y=list(cex=1.4)))

lm variable importance

                         Overall
realMELDNa               100.000
intermedLactates          99.531
surgeryDuration           75.655
noradrenalineEndSurger    73.023
Albumina50Comp            64.885
degenzaPreTra             64.539
ratioPaO2FiO2             59.905
organDonorDrawalSodiemia  43.986
albumina                  40.763
plasmaPools               29.857
organDonorHeight          28.258
platCountTranspl          22.170
organDonorGammaGT         17.322
CristalloidiComp          13.998
doseFibrinogen             7.761
plasmaFresco               4.550
organDonorAge              2.259
CRIOprecipitated           0.000

In [22]:
test_select$predicted <- predict(lm_select, newdata = test_select)
cor(test_select$degenzaPostTraOSP ,test_select$predicted)^2  #rsquared

[1] 0.2579623

In [23]:
mae(test_select$degenzaPostTraOSP ,test_select$predicted)

[1] 0.425212

In [24]:
rmse(test_select$degenzaPostTraOSP, test_select$predicted)

[1] 0.5605844

In [25]:
#predicted vs actual

ggplot(data = test_select, aes(x = predicted, y = degenzaPostTraOSP)) +
  geom_point(alpha = 0.2, color = "blue") +
  geom_smooth(aes(x = predicted,
                  y = degenzaPostTraOSP), color="blue") +
  geom_line(aes(x = degenzaPostTraOSP,
                y = degenzaPostTraOSP), color = "black", 
            linetype = 2, size = 1)

`geom_smooth()` using method = 'loess' and formula 'y ~ x'



In [26]:
# residuals

ggplot(data = test_select, aes(x = predicted,
                      y = predicted - degenzaPostTraOSP)) +
  geom_point(alpha = 0.2, color = "blue") +
  geom_smooth(aes(x = predicted,
                  y = predicted - degenzaPostTraOSP),
              color="black")

`geom_smooth()` using method = 'loess' and formula 'y ~ x'

